# Political Organizations Activity on Twitter aggregated by Party

The parameters in the cell below can be adjusted to explore other political parties and time frames.

### How to explore other political parties?
The ***organization*** parameter can be use to aggregate organizations by their party. The column `subcategory` in this [this other notebook](../political_organizations.ipynb?autorun=true) show the organizations that belong each party.

***Alternatively***, you can direcly use the [organizations API](http://mediamonitoring.gesis.org/api/organizations/swagger/), or access it with the [SMM Wrapper](https://pypi.org/project/smm-wrapper/).

## A. Set Up parameters

In [ ]:
# Parameters: 
organization = 'csu'
from_date = '2017-09-01'
to_date = '2018-12-31'
aggregation = 'week'

## B. Using the SMM Organization API

In [ ]:
library("httr")
library("jsonlite")
library('dplyr', warn.conflicts = FALSE)

# prepare urls
base <- "http://mediamonitoring.gesis.org/api/organizations/"
url_all <- paste(base,"all/", sep="")
url_tweets <- paste(base, "twitter/tweets_by/organizations/", sep="")
url_replies <- paste(base, "twitter/replies_to/organizations/", sep="")

# prepare parameters
params = list(
    from_date=from_date,
    to_date=to_date,
    aggregate_by=aggregation
)

# use the api to get the organizations and filter the parties
orgs <- as.data.frame(fromJSON(content(GET(url_all), "text", encoding="UTF-8"), flatten = TRUE))
party_df <- orgs[orgs$subcategory == organization & !is.null(orgs$tw_ids),]

# query the Social Media Monitoring API for tweets and replies
tweets <- data.frame()
replies <- data.frame()
for (organization_id in party_df$organization_id) {
    json_tweets <- fromJSON(content(GET(paste(url_tweets, organization_id, "/?", sep=""),query=params), "text", encoding="UTF-8"), flatten = TRUE)
    json_replies <- fromJSON(content(GET(paste(url_replies, organization_id, "/?", sep=""), query=params), "text", encoding="UTF-8"), flatten = TRUE)

    # concatenate
    if (length(json_tweets$values) != 0) {
        tweets <- rbind(tweets, as.data.frame(json_tweets))
    }    
    if (length(json_replies$values) != 0) {
        replies <- rbind(replies, as.data.frame(json_replies))
    }
}

# group by day, week, or month, and then merge
tweets <- summarise(group_by(tweets, labels), tweets = sum(values), response_type = aggregation)
replies <- summarise(group_by(replies, labels), replies = sum(values), response_type = aggregation)
merged <- merge(tweets, replies, by='labels')

## C. Plotting

In [ ]:
library("ggplot2")

#plotting
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = merged, mapping = aes(as.Date(labels))) +
  geom_line(aes(y = tweets, color="Tweets", group=response_type.x)) +
  geom_line(aes(y = replies, color="Replies", group=response_type.y)) +
  labs(title = "Twitter (tweets)", y = "Tweets") +
  theme(axis.text.x = element_text(angle = 60, hjust = 0.5, vjust = 0.5),
        axis.title.x = element_blank(), legend.title = element_blank()) +
  scale_x_date(date_breaks = "1 month")